# BANK TICKET CANCELLATION RATE ANALYSIS

#### Objective:
* Clean the orders data, filter outliers
* Study order cancellation rate, its characteristics and dynamics
* Analyze cancellation rate of orders paid with bank tickets (boletos), its characteristics and dynamics
* Perform a root cause analysis of bank ticket cancellations

#### Inputs:
* Orders fact (+ detailed to Order items)
* Customers dimension
* Products dimension

#### Output:
* Graphs, tables and other elements to be used in root cause analysis presentation

#### Data:
* Time Span: 2016-01-01 through 2016-06-30

## Python settings

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# Jupyter notebook settings
sns.set_context('notebook')
sns.set_style('darkgrid')

# I. Data Preparation
# 1. Data preparation - Orders

## 1.1. Select data from databases - Orders

In [2]:
# read the .db file
orders_cnx = sqlite3.connect('databases/Orders.db')

In [3]:
# select data from databases and set to respective dataframes
df_raw = pd.read_sql_query("SELECT * FROM Orders", orders_cnx)

In [4]:
df_raw.shape

(61499, 34)

In [5]:
df_raw.columns

Index(['id', 'store_id', 'user_id', 'order_date', 'order_raw', 'device_type',
       'order_channel_code_online', 'order_retail_type', 'order_status',
       'order_source_and_device', 'payment_method', 'payment_num_installments',
       'payment_coupon', 'shipping_method_id', 'address_city',
       'address_region', 'address_state', 'address_zip', 'ip_address',
       'is_mobile', 'is_placed_order', 'is_shipped_order',
       'days_until_next_placed_order', 'is_shipped_purchase_of_new_buyer',
       'is_purchase_of_new_buyer', 'utm_medium', 'utm_source', 'utm_campaign',
       'shipping_paid_by_amaro', 'shipping_paid_by_customer', 'order_subtotal',
       'order_discount', 'shipping_cost', 'order_total'],
      dtype='object')

Column names are already in appropriate format (lowercase, underscores instead of spaces), no need for renaming.

## 1.2. Format data - Orders 

In [6]:
# glance at the data to identify formatting needs
df1 = df_raw.copy()
df1.head().transpose()

,0,1,2,3,4
id,1594409,1652797,1754030,3789151,1582210
store_id,2,4,1,1,2
user_id,82,82,772,772,864
order_date,2016-02-02,2016-02-21,2016-03-23,2016-04-04,2016-01-29
order_raw,2016-02-02 17:34:30,2016-02-21 17:02:24,2016-03-23 23:13:38,2016-04-04 00:00:00,2016-01-29 16:05:04
device_type,desktop,desktop,desktop,,desktop
order_channel_code_online,GSL,GSL,WEB,TRO,WEB
order_retail_type,Physical,Physical,Online,Online,Online
order_status,DELIVERED,DELIVERED,DELIVERED,DELIVERED,DELIVERED
order_source_and_device,desktop web,desktop web,desktop web,millennium,desktop web


In [7]:
df1.dtypes

id                                  object
store_id                            object
user_id                             object
order_date                          object
order_raw                           object
device_type                         object
order_channel_code_online           object
order_retail_type                   object
order_status                        object
order_source_and_device             object
payment_method                      object
payment_num_installments            object
payment_coupon                      object
shipping_method_id                  object
address_city                        object
address_region                      object
address_state                       object
address_zip                         object
ip_address                          object
is_mobile                           object
is_placed_order                     object
is_shipped_order                    object
days_until_next_placed_order        object
is_shipped_

In [8]:
# date format
df1['order_date'] = pd.to_datetime(df1['order_date'])
df1['order_raw'] = pd.to_datetime(df1['order_raw'])

# number format
df1['id'] = pd.to_numeric(df1['id'])
#df1['store_id'] = pd.to_numeric(df1['store_id'])
df1['user_id'] = pd.to_numeric(df1['user_id'])
df1['days_until_next_placed_order'] = pd.to_numeric(df1['days_until_next_placed_order'])
df1['shipping_paid_by_amaro'] = pd.to_numeric(df1['shipping_paid_by_amaro'])
df1['shipping_paid_by_customer'] = pd.to_numeric(df1['shipping_paid_by_customer'])
df1['order_subtotal'] = pd.to_numeric(df1['order_subtotal'])
df1['order_discount'] = pd.to_numeric(df1['order_discount'])
df1['shipping_cost'] = pd.to_numeric(df1['shipping_cost'])
df1['order_total'] = pd.to_numeric(df1['order_total'])

# split columns into categorical and numerical
num_features = df1.select_dtypes(include=['int64', 'float64', 'datetime64'])
cat_features = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64'])

# lower case categorical features
df1[cat_features.columns] = df1[cat_features.columns].apply(lambda x: x.astype('str').str.lower()).replace(' ', '_', regex=True)

#check the result
df1.dtypes

id                                           int64
store_id                                    object
user_id                                      int64
order_date                          datetime64[ns]
order_raw                           datetime64[ns]
device_type                                 object
order_channel_code_online                   object
order_retail_type                           object
order_status                                object
order_source_and_device                     object
payment_method                              object
payment_num_installments                    object
payment_coupon                              object
shipping_method_id                          object
address_city                                object
address_region                              object
address_state                               object
address_zip                                 object
ip_address                                  object
is_mobile                      

# 2. Data selection - Orders

## 2.1 Filter rows - Orders 

In [9]:
# remove 'Devolução & Troca' orders from analysis
df1 = df1[~df1['payment_method'].isin(['devolução_troca'])]
# remove AMAROteam members' orders
df1 = df1[~df1['payment_coupon'].isin(['AMAROTEAM'])]
# remove duplicated
df1 = df1.drop_duplicates(subset='id')

## 2.2 Select columns  - Orders

### Columns skipped
1. **store_id:** No analysis made taking Guide Shops into consideration 
2. **order_date:** Already part of Order Raw - no need to use Order Date  
3. **device_type:** No analysis based taking Device Type into consideration
4. **order_channel_code_online:** No analysis based taking Order Channel Code Online into consideration
5. **order_retail_type:** No analysis based taking Order Retail Type into consideration
6. **order_source_and_device:** No analysis based taking Order Source and Device into consideration
7. **payment_num_installments:** No analysis based taking Number of installments into consideration
8. **payment_coupon:** Payment coupon of team members excluded
9. **shipping_method_id:** ID of shipping method is redundant
10. **address_city:** The lowest level of granularity in geographical analysis is State  
11. **address_region:** The lowest level of granularity in geographical analysis is State  
12. **address_zip:** The lowest level of granularity in geographical analysis is State  
13. **ip_address:** Not necessary when analysing the orders
14. **is_mobile:** Not taken into consideration in the analysis
15. **is_placed_order:** Already makes part of 'order_status' that we'll analyze
16. **is_shipped_order:** Already makes part of 'order_status' that we'll analyze
17. **is_shipped_purchase_of_new_buyer:** Isn't taken into consideration
18. **is_purchase_of_new_buyer:** Isn't taken into consideration
19. **utm_medium:** No analysis based taking UTM Medium into consideration
20. **utm_source:** No analysis based taking UTM Source into consideration
21. **utm_campaign:** No analysis based taking UTM Campaign into consideration
22. **shipping_paid_by_amaro:** Use shipping_cost, doesn't matter if paid by customer or by AMARO
23. **shipping_paid_by_customer:** Use shipping_cost, doesn't matter if paid by customer or by AMARO


In [10]:
# select columns for analysis
cols = ['id','user_id', 'order_raw', 'order_status', 'payment_method', 'address_state',
               'days_until_next_placed_order', 'order_subtotal', 'order_discount', 'shipping_cost', 'order_total']
# select categorical columns for aggregation
cols_agg = cols[:-4]
df2 = df1[cols]

# 3. Aggregation - Orders

In [11]:
df_orders = df2.groupby(cols_agg).sum().reset_index()

In [12]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43751 entries, 0 to 43750
Data columns (total 11 columns):
id                              43751 non-null int64
user_id                         43751 non-null int64
order_raw                       43751 non-null datetime64[ns]
order_status                    43751 non-null object
payment_method                  43751 non-null object
address_state                   43751 non-null object
days_until_next_placed_order    43751 non-null float64
order_subtotal                  43751 non-null float64
order_discount                  43751 non-null float64
shipping_cost                   43751 non-null float64
order_total                     43751 non-null float64
dtypes: datetime64[ns](1), float64(5), int64(2), object(3)
memory usage: 3.7+ MB


# 1. Data Preparation - Order Items
## 1.1. Select data from databases - Order Items

In [13]:
# read the .db file
oi_cnx = sqlite3.connect('databases/OrderItems.db')

# select data from databases and set to respective dataframes
df_raw = pd.read_sql_query("SELECT * FROM OrderItems", oi_cnx)

In [14]:
df_raw.shape

(136774, 8)

In [15]:
df_raw.columns

Index(['order_item_id', 'order_id', 'sku', 'promotional_price',
       'original_price', 'items_quantity', 'product_cost',
       'is_item_sold_with_discount'],
      dtype='object')

## 1.2. Format data - Order Items

In [16]:
# glance at the data to identify formatting needs
df1 = df_raw.copy()
df1.head()

,order_item_id,order_id,sku,promotional_price,original_price,items_quantity,product_cost,is_item_sold_with_discount
0,328703,1594409,2195_73_0_M,257.98,351.88,1,79.53,t
1,328702,1594409,2881_147_0_M,211.03,281.46,1,102.44,t
2,328700,1594409,4097_263_0_G,175.82,234.51,1,75.10,t
3,328701,1594409,5403_1000026_0_U,117.14,117.14,1,37.54,f
4,342375,1652797,2967_1000058_0_M,174.23,348.72,1,117.15,t


In [17]:
df1.dtypes

order_item_id                 object
order_id                      object
sku                           object
promotional_price             object
original_price                object
items_quantity                object
product_cost                  object
is_item_sold_with_discount    object
dtype: object

### 1.2.1. Parse data types

In [18]:
df1['order_item_id'] = pd.to_numeric(df1['order_item_id'])
df1['order_id'] = pd.to_numeric(df1['order_id'])
df1['promotional_price'] = pd.to_numeric(df1['promotional_price'])
df1['original_price'] = pd.to_numeric(df1['original_price'])
df1['items_quantity'] = pd.to_numeric(df1['items_quantity'])
df1['product_cost'] = pd.to_numeric(df1['product_cost'])
#df1['is_item_sold_with_discount'] = pd.to_numeric(df1['is_item_sold_with_discount']) #boolean(?)

### 1.2.2. Create new columns

In [19]:
# create Size column based on the last element in SKU
df1['size'] = df1['sku'].apply(lambda x: x.split('_')[-1])

# create discount column:
# Difference between Original and Promotional price when 'Is item sold with discount' is True, and 0 otherwise
conditions = [df1['is_item_sold_with_discount'] == 't', df1['is_item_sold_with_discount'] == 'f']

outputs_discount = [df1['original_price'] - df1['promotional_price'],0]
outputs_discount_share = [(df1['original_price'] - df1['promotional_price'])/df1['original_price'],0]
df1['discount'] = np.select(conditions, outputs_discount, 0)
df1['discount_share'] = np.select(conditions, outputs_discount_share, 0)

# create profit column:
# difference between Promotional price and Product cost when 'Is item sold with discount' is True, 
# difference between Original price and Product cost otherwise
outputs_profit = [df1['promotional_price'] - df1['product_cost'],df1['original_price'] - df1['product_cost']]
outputs_profit_margin = [(df1['promotional_price'] - df1['product_cost'])/df1['product_cost'],(df1['original_price'] - df1['product_cost'])/df1['product_cost']]
df1['profit'] = np.select(conditions, outputs_profit, 0)
df1['profit_margin'] = np.select(conditions, outputs_profit_margin, 0)
df1.head(20)

,order_item_id,order_id,sku,promotional_price,original_price,items_quantity,product_cost,is_item_sold_with_discount,size,discount,discount_share,profit,profit_margin
0,328703,1594409,2195_73_0_M,257.98,351.88,1,79.53,t,M,93.90,0.266852,178.45,2.243807
1,328702,1594409,2881_147_0_M,211.03,281.46,1,102.44,t,M,70.43,0.250231,108.59,1.060035
2,328700,1594409,4097_263_0_G,175.82,234.51,1,75.10,t,G,58.69,0.250267,100.72,1.341145
3,328701,1594409,5403_1000026_0_U,117.14,117.14,1,37.54,f,U,0.00,0.000000,79.60,2.120405
4,342375,1652797,2967_1000058_0_M,174.23,348.72,1,117.15,t,M,174.49,0.500373,57.08,0.487239
5,342374,1652797,4087_261_0_M,111.92,224.09,1,63.02,t,M,112.17,0.500558,48.90,0.775944
6,342373,1652797,4393_275_0_P,199.16,398.57,1,115.54,t,P,199.41,0.500314,83.62,0.723732
7,365786,1754030,2869_149_0_M,161.77,273.94,1,69.69,t,M,112.17,0.409469,92.08,1.321280
8,365784,1754030,4097_255_0_M,149.31,249.01,1,79.74,t,M,99.70,0.400386,69.57,0.872460
9,365790,1754030,4097_257_0_M,149.31,249.01,1,79.74,t,M,99.70,0.400386,69.57,0.872460


### 1.2.3. Split into categorical and numerical

In [20]:
# split columns into categorical and numerical
num_features = df1.select_dtypes(include=['int64', 'float64', 'datetime64'])
cat_features = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64'])

# lower case categorical features
df1[cat_features.columns] = df1[cat_features.columns].apply(lambda x: x.astype('str').str.lower()).replace(' ', '_', regex=True)
df1.dtypes

order_item_id                   int64
order_id                        int64
sku                            object
promotional_price             float64
original_price                float64
items_quantity                  int64
product_cost                  float64
is_item_sold_with_discount     object
size                           object
discount                      float64
discount_share                float64
profit                        float64
profit_margin                 float64
dtype: object

# 2. Data selection - Order Items
## 2.1 Filter rows - Order Items

In [21]:
# Payment method 'Devolução & Troca' & coupon 'AMAROTEAM' removed from Orders, need to remove from Order Items as well 
df1 = df1[df1['order_id'].isin(list(df_orders['id']))]

# remove duplicated
df1 = df1.drop_duplicates(subset='order_item_id')

## 2.2 Select columns - Order Items
### Columns skipped
1. **sku:** No other utility of the sku for analysis apart from item size, for which there's already 'Size' column
2. **is item sold with discount:** Already used for defining rules for discounts, not needed otherwise

In [22]:
# select columns for analysis, in a new order so that numerical variables follow categorical
cols = ['order_item_id', 'order_id', 'size', 'items_quantity', 'promotional_price',
       'original_price',  'product_cost', 'discount', 'discount_share', 'profit', 'profit_margin']
# select categorical columns for aggregation
cols_agg = cols[:-8]
df2 = df1[cols]

# 3. Aggregation - Order Items

In [23]:
#how to aggregate partially summing and partially averaging?
df_order_items = df2.groupby(cols_agg).mean().reset_index()

# 1. Data preparation - Customers
## 1.1. Select data from databases - Customers

In [24]:
# read the .db file
customers_cnx = sqlite3.connect('databases/Customers.db')
# select data from database
df_raw = pd.read_sql_query("SELECT * FROM Customers", customers_cnx)

In [25]:
df_raw.shape

(30293, 21)

In [26]:
df_raw.columns

Index(['user_id', 'account_created_date', 'age', 'gender', 'signup_channel',
       'signup_device_type', 'signup_ip_address', 'signup_store_id',
       'shipping_address_country', 'shipping_address_state',
       'shipping_address_city', 'shipping_address_zip', 'birthdate_date',
       'first_shipped_order_device_type',
       'first_shipped_order_order_channel_code_online',
       'first_shipped_order_order_retail_type_level_2',
       'first_shipped_order_date', 'first_placed_order_date',
       'first_placed_order_device_type',
       'first_placed_order_order_channel_code_online',
       'first_placed_order_order_retail_type_level_2'],
      dtype='object')

## 1.2. Format data - Customers

In [27]:
# glance at the data to identify formatting needs
df1 = df_raw.copy()
df1.head()

,user_id,account_created_date,age,gender,signup_channel,signup_device_type,signup_ip_address,signup_store_id,shipping_address_country,shipping_address_state,...,shipping_address_zip,birthdate_date,first_shipped_order_device_type,first_shipped_order_order_channel_code_online,first_shipped_order_order_retail_type_level_2,first_shipped_order_date,first_placed_order_date,first_placed_order_device_type,first_placed_order_order_channel_code_online,first_placed_order_order_retail_type_level_2
0,89256,,35,,,,,0,Brasil,SP,...,01535000,1983-05-28,desktop,WEB,Online,2016-03-06,2016-03-06,desktop,WEB,Online
1,88350,,44,,,,,0,Brasil,SP,...,18080709,1974-09-10,,WEB,Online,2015-09-06,2015-09-06,,WEB,Online
2,89688,,57,,,,,0,Brasil,RS,...,98995000,1961-12-28,desktop,WEB,Online,2016-06-23,2016-06-23,desktop,WEB,Online
3,90364,,61,,,,,0,Brasil,SP,...,16050640,1958-01-22,desktop,WEB,Online,2016-03-11,2016-03-11,desktop,WEB,Online
4,127080,,34,F,,,,0,Brasil,SP,...,04542001,1984-11-22,desktop,WEB,Online,2015-12-04,2013-07-18,,-,Online


In [28]:
# date format
df1['account_created_date'] = pd.to_datetime(df1['account_created_date'])
df1['birthdate_date'] = pd.to_datetime(df1['birthdate_date'])
df1['first_shipped_order_date'] = pd.to_datetime(df1['first_shipped_order_date'])
df1['first_placed_order_date'] = pd.to_datetime(df1['first_placed_order_date'])

# number format
df1['user_id'] = pd.to_numeric(df1['user_id'])
df1['age'] = pd.to_numeric(df1['age'])
df1['signup_store_id'] = pd.to_numeric(df1['signup_store_id'])
df1['shipping_address_zip'] = pd.to_numeric(df1['shipping_address_zip'])

# split columns into categorical and numerical
num_features = df1.select_dtypes(include=['int64', 'float64', 'datetime64'])
cat_features = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64'])

# lower case categorical features
df1[cat_features.columns] = df1[cat_features.columns].apply(lambda x: x.astype('str').str.lower()).replace(' ', '_', regex=True)
df1.columns

Index(['user_id', 'account_created_date', 'age', 'gender', 'signup_channel',
       'signup_device_type', 'signup_ip_address', 'signup_store_id',
       'shipping_address_country', 'shipping_address_state',
       'shipping_address_city', 'shipping_address_zip', 'birthdate_date',
       'first_shipped_order_device_type',
       'first_shipped_order_order_channel_code_online',
       'first_shipped_order_order_retail_type_level_2',
       'first_shipped_order_date', 'first_placed_order_date',
       'first_placed_order_device_type',
       'first_placed_order_order_channel_code_online',
       'first_placed_order_order_retail_type_level_2'],
      dtype='object')

# 2. Data selection - Customers
## 2.1 Filter rows - Customers

In [29]:
# Remove customers that weren't placing orders featured in given dataset
df1 = df1[df1['user_id'].isin(list(df_orders['user_id']))]

# remove duplicated
df1 = df1.drop_duplicates(subset='user_id')

## 2.2 Select columns - Customers
**Columns skipped - All columns except for**
1. **user_id:** Table key
2. **account_created date:** Necessary to know customer's lifetime
3. **age:** Necessary for clustering customers
4. **shipping_address_state:** Necessary for clustering customers

In [30]:
# select columns for analysis, in a new order so that numerical variables follow categorical
cols = ['user_id', 'account_created_date', 'age', 'shipping_address_state']

In [31]:
df_customers = df1[cols]
df_customers

,user_id,account_created_date,age,shipping_address_state
0,89256,NaT,35.0,sp
1,88350,NaT,44.0,sp
2,89688,NaT,57.0,rs
3,90364,NaT,61.0,sp
5,122486,NaT,43.0,pr
6,135914,NaT,54.0,pr
7,133832,NaT,32.0,sp
8,139254,NaT,43.0,df
9,136686,NaT,48.0,rs
10,139576,NaT,54.0,pr


#describe data print nummber of rows, number of columns
2.1. Data dimensions
2.2. Data type attributes - double check
2.3. number of NAs isna()..sum - more than 70% of data null - jogar fora, , mas ate 10% null - fill na
2.4. Descriptive Statistics - tendencia e dispersao central tendency and dispersion - separate numerical and categorical, media, mediana, moda, stdev, range, kurtosis, skew, 
separately for 
categorical - median, 1st quartile, QR, min, max, outliers, - boxplot, 

# II. DATA ANALYSIS
# 1. High level description
## 1.1. Data Dimensions

In [32]:
print('=== Orders ===')
print('Number of Rows: {}'.format(df_orders.shape[0]))
print('Number of Columns: {}\n'.format(df_orders.shape[1]))

print('=== Orders - Column types ===')
print(df_orders.dtypes)

=== Orders ===
Number of Rows: 43751
Number of Columns: 11

=== Orders - Column types ===
id                                       int64
user_id                                  int64
order_raw                       datetime64[ns]
order_status                            object
payment_method                          object
address_state                           object
days_until_next_placed_order           float64
order_subtotal                         float64
order_discount                         float64
shipping_cost                          float64
order_total                            float64
dtype: object


In [33]:
print('=== Order items ===')
print('Number of Rows: {}'.format(df_order_items.shape[0]))
print('Number of Columns: {}\n'.format(df_order_items.shape[1]))

print('=== Order items - Column types ===')
print(df_order_items.dtypes)

=== Order items ===
Number of Rows: 101913
Number of Columns: 11

=== Order items - Column types ===
order_item_id          int64
order_id               int64
size                  object
items_quantity         int64
promotional_price    float64
original_price       float64
product_cost         float64
discount             float64
discount_share       float64
profit               float64
profit_margin        float64
dtype: object


In [34]:
print('=== Customers ===')
print('Number of Rows: {}'.format(df_customers.shape[0]))
print('Number of Columns: {}\n'.format(df_customers.shape[1]))

print('=== Customers - Column types ===')
print(df_customers.dtypes)

=== Customers ===
Number of Rows: 19807
Number of Columns: 4

=== Customers - Column types ===
user_id                            int64
account_created_date      datetime64[ns]
age                              float64
shipping_address_state            object
dtype: object


## 1.2. Data type attributes - categorical features

In [35]:
c = pd.DataFrame(cat_features.apply(lambda x: x.unique().shape[0], axis=0).reset_index())
c.columns = ['Columns Names', 'Number of Levels']
print(c)

                                    Columns Names  Number of Levels
0                                          gender                 3
1                                  signup_channel                 3
2                              signup_device_type                 3
3                               signup_ip_address              9935
4                        shipping_address_country                 2
5                          shipping_address_state                28
6                           shipping_address_city              1847
7                 first_shipped_order_device_type                 4
8   first_shipped_order_order_channel_code_online                10
9   first_shipped_order_order_retail_type_level_2                 3
10                 first_placed_order_device_type                 4
11   first_placed_order_order_channel_code_online                12
12   first_placed_order_order_retail_type_level_2                 3


## 1.3. Data type attributes - numerical features

## 1.4. Number of NAs

In [36]:
df_orders.isnull().sum(axis=0)
#if more than 70% are null - throw the columns away, 10% null - use one of the strategies of fill na

id                              0
user_id                         0
order_raw                       0
order_status                    0
payment_method                  0
address_state                   0
days_until_next_placed_order    0
order_subtotal                  0
order_discount                  0
shipping_cost                   0
order_total                     0
dtype: int64

In [37]:
df_order_items.isnull().sum(axis=0)
#if more than 70% are null - throw the columns away, 10% null - use one of the strategies of fill na
#1) ignorar os nas, fazer analysis assim msm
#2) importance of the column, tirar a coluna inteira
#3) if important, see how many lines have the nas, 
df_order_items.isnull().sum(axis=0)
#here only identifying - and later on in the univeriate analysis for promotional price do the discovery 
#SHOW THE PERCENTAGE

order_item_id          0
order_id               0
size                   0
items_quantity         0
promotional_price    586
original_price       202
product_cost           0
discount               2
discount_share         3
profit                 2
profit_margin          2
dtype: int64

Very few values are NA, doesn't make sense to use the fill NA strategies

## 1.5. Data Description

In [38]:
num_features = df_orders.select_dtypes(include=['int64', 'float64', 'datetime64'])
cat_features = df_orders.select_dtypes(exclude=['int64', 'float64', 'datetime64'])

df_orders[num_features.columns].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
id,43751.0,2.050750e+06,6.027378e+05,1514142.00,1700256.50,1887603.00,2085427.00,3987250.00
user_id,43751.0,2.254360e+07,1.471000e+07,28.00,551740.00,32776152.00,32893604.00,33277134.00
days_until_next_placed_order,43751.0,1.276105e+02,2.119701e+02,0.00,4.00,28.00,147.00,1124.00
order_subtotal,43751.0,6.369239e+02,6.788671e+02,18.78,281.46,460.64,747.53,30684.98
order_discount,43751.0,4.022198e+01,2.461560e+02,0.00,0.00,0.00,0.00,8929.29
shipping_cost,43751.0,2.210238e+01,1.447539e+01,0.00,9.90,23.24,33.20,384.91
order_total,43751.0,2.499033e+02,2.594049e+02,0.00,119.65,184.62,297.24,13071.90


In [39]:
df_orders[cat_features.columns].describe().transpose()

,count,unique,top,freq
order_status,43751,5,delivered,32241
payment_method,43751,51,cartão,23868
address_state,43751,29,sp,24143


# 2. EDA - Exploratory Data Analysis

In [40]:
df_orders.head()

,id,user_id,order_raw,order_status,payment_method,address_state,days_until_next_placed_order,order_subtotal,order_discount,shipping_cost,order_total
0,1514142,32719850,2016-01-01 01:52:03,delivered,cartão,sp,4.0,224.09,0.0,7.01,96.91
1,1514153,32753096,2016-01-01 11:04:36,delivered,cartão,pa,34.0,472.84,0.0,53.22,189.70
2,1514164,7535066,2016-01-01 11:05:18,cancelled,boleto,mg,2.0,149.31,0.0,13.32,73.22
3,1514186,32807492,2016-01-01 11:39:55,delivered,boleto,sp,4.0,249.01,0.0,10.05,109.95
4,1514197,32823862,2016-01-01 12:07:10,delivered,cartão,rj,19.0,373.64,0.0,9.12,159.02


## 2.1. Univariate Analysis